# Comparing memory usage for 3 packages: `pytraj`, `mdtraj`, `MDAnalysis`

#### Hints: `pytraj` is 3 times less efficient in memory than `mdtraj`. `MDAnalysis` uses iterator all the time so it's very friendly with RAM

In [1]:
# require: ipython_memory_usage.py (https://github.com/ianozsvald/ipython_memory_usage) # to profile
# every line of code in ipython and notebook
# https://pypi.python.org/pypi/memory_profiler
# https://pypi.python.org/pypi/psutil

# Note: I turned off MDAnalysis since getting error about netcdf file when running this notebook several times.

In [2]:
import netCDF4 as netcdf
import ipython_memory_usage as imu
import mdtraj as md
import pytraj.io as io
from MDAnalysis import Universe
imu.start_watching_memory()

In [2] used 39.2578 MiB RAM in 0.97s, peaked 0.00 MiB above current, total RAM usage 74.16 MiB


## Loading

In [3]:
## loading to memory
topname = "../tests/data/nogit/remd/myparm.top"
fname = "../tests/data/nogit/remd/remd.000.ncdf"

!du -sh $fname

200M	../tests/data/nogit/remd/remd.000.ncdf
In [3] used 0.0586 MiB RAM in 0.22s, peaked 0.00 MiB above current, total RAM usage 74.21 MiB


In [4]:
from pytraj import Trajectory
fa = Trajectory(fname, topname)

In [4] used 1080.5820 MiB RAM in 2.70s, peaked 0.00 MiB above current, total RAM usage 1154.80 MiB


In [5]:
# load mdtraj
mtraj = md.load(fname, top="../tests/data/nogit/remd/myparm.parm7")

In [5] used 219.1250 MiB RAM in 2.12s, peaked 199.68 MiB above current, total RAM usage 1373.92 MiB


In [6]:
# load MDAnalysis
u = Universe(topname, fname)

In [6] used 12.0430 MiB RAM in 1.06s, peaked 0.00 MiB above current, total RAM usage 1385.96 MiB


In [7]:
# load pytraj: TrajReadOnly
trajread = io.iterload(fname, topname)

In [7] used 12.0898 MiB RAM in 0.31s, peaked 0.00 MiB above current, total RAM usage 1398.05 MiB


In [8]:
# load pytraj: Trajectory
fa = io.load(fname, topname)[:]

# Trajectory is 2 times heavier than mtraj (from mdtraj), both trajectories are in memory. The file size is ~200 MB.
# Trajectory is too fat. :D

In [8] used 535.5664 MiB RAM in 2.08s, peaked 0.00 MiB above current, total RAM usage 1933.62 MiB


In [9]:
# load pytraj: Trajectory
traj = io.load(fname, topname)
fa = traj[:]

# Trajectory is 2 times heavier than mtraj (from mdtraj), both trajectories are in memory. The file size is ~200 MB.
# Trajectory is too fat. :D

In [9] used 535.4062 MiB RAM in 2.08s, peaked 0.00 MiB above current, total RAM usage 2469.03 MiB


## Slicing - single index

In [10]:
# MDAnalysis
u.trajectory[100]

< Timestep 101 with unit cell dimensions [  61.17625427   61.17625427   61.17625427  109.47122192  109.47122192
  109.47122192] >

In [10] used 0.0703 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 2469.10 MiB


In [11]:
# mdtraj
mtraj[100]

<mdtraj.Trajectory with 1 frames, 17443 atoms, 5666 residues, and unitcells at 0x2aab284510d0>

In [11] used 9.7773 MiB RAM in 0.46s, peaked 0.00 MiB above current, total RAM usage 2478.88 MiB


In [12]:
# pytraj
trajread[100]

<Frame with 17443 atoms>

In [12] used 0.4258 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 2479.30 MiB


In [13]:
# pytraj, Trajectory
fa[100]

<Frame with 17443 atoms>

In [13] used 0.1445 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 2479.45 MiB


## Slicing - a chunk

In [14]:
# MDanalysis, they use iterator. memory is very small
u.trajectory[200:500]

<generator object iterNETCDF at 0x2aab1832b690>

In [14] used 0.0039 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 2479.45 MiB


In [15]:
# mdtraj
mtraj[200:500]

<mdtraj.Trajectory with 300 frames, 17443 atoms, 5666 residues, and unitcells at 0x2aab188cc9d0>

In [15] used 70.2109 MiB RAM in 0.58s, peaked 0.00 MiB above current, total RAM usage 2549.66 MiB


In [16]:
# pytraj, TrajReadOnly
trajread[200:500]

<pytraj.Trajectory with 300 frames: <Topology with 5634 mols, 5666 residues, 17443 atoms, 17452 bonds, PBC with box type = truncoct>>
           

In [16] used 158.4531 MiB RAM in 0.64s, peaked 0.00 MiB above current, total RAM usage 2708.11 MiB


In [17]:
# pytraj, Trajectory, very little memory since we only created `view`
fa[200:500]

<pytraj.Trajectory with 300 frames: <Topology with 5634 mols, 5666 residues, 17443 atoms, 17452 bonds, PBC with box type = truncoct>>
           

In [17] used 0.1250 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 2708.24 MiB


# Conclusion
* MDanalysis is the best friend of RAM
* mdtraj: 2nd
* pytraj: Trajectory is too fat (is there anyway to have both supperfast iterating + less memory?). But we have TrajectoryIterator two